## Demand Survey ##

The idea for this mini-lab is to derive a Demand curve from student input via a Google Form.  


The Survey is here
https://forms.gle/XzaNkbJHSYUyKscSA


This notebook reads info directly from a google spreadsheet using a google API key hosted within the Jupyterhub server:
https://docs.datahub.berkeley.edu/en/latest/services/google-sheets.html


In [ ]:
from gspread_pandas.client import Spread
import os
import json
from oauth2client.service_account import ServiceAccountCredentials
from datascience import *
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Authenticate to Google
scope = ['https://spreadsheets.google.com/feeds', 'https://www.googleapis.com/auth/drive']
creds = ServiceAccountCredentials.from_json_keyfile_dict(json.loads(os.environ['GOOGLE_SHEETS_READONLY_KEY']), scope)

In [ ]:
#This line reads in data from a google sheet that is the responses to the survey form
url = 'https://docs.google.com/spreadsheets/d/1Ias5oYZIQCQgVbjL-2jeeo04-tyg_4-hhPZZFPRFO1U/edit#gid=1350374019'

In [ ]:
# Open the Google Sheet, and print contents of sheet 1 as a dataframe
spread = Spread(creds, url)
sheet_df = spread.sheet_to_df(sheet='Form Responses 1',header_rows=1,index=0)
print(sheet_df)
print (sheet_df.dtypes)

In [ ]:
previous_df=sheet_df.iloc[0:25]
#previous_df

In [ ]:
class_df=sheet_df.iloc[25:]
#class_df

In [ ]:
# Looks like they imported as Strings, lets make them numeric...
cols = sheet_df.columns.drop('Timestamp')
sheet_df[cols] = sheet_df[cols].apply(pd.to_numeric, errors='coerce')

In [ ]:
# make a datascience Table from Pandas DF
DemandTable=Table.from_df(sheet_df)
DemandTable

In [ ]:
DemandTable.hist('GTKitkat')
DemandTable.hist('Burrito')
DemandTable.hist('GreekTix')
DemandTable.hist('Iphone12')

Lets start by looking just at Kitkats


And let't try to figure how how many people would buy GTKitkats at a given price? Let's assume that a person would be willing to buy the good at a price less than their bid price.  

In [ ]:
KitkatsTable=DemandTable.select('GTKitkat')
KitkatsTable

In [ ]:
# Lets make a vector of prices
DemandGTK= Table().with_columns([
    'priceGTK',[0.25, 0.5, 0.75, 1.00, 1.25, 1.5,1.75,2],    
])
DemandGTK

In [ ]:
# Count how many people are in each answer pool
KitkatsTable.group("GTKitkat")

In [ ]:
#Cumulative Sum of how many people are willing to pay at a agiven price
Qdemand = np.flip(np.cumsum(np.flip(KitkatsTable.group("GTKitkat").column("count"))))

In [ ]:
# # Lets count how many people would buy at a given price
## This is not the right way to code!
# Qdemand=np.count_nonzero(KitkatsTable.column('GTKitkat') <= 2)
# Qdemand=np.append(Qdemand,np.count_nonzero(KitkatsTable.column('GTKitkat') <= 1.75))
# Qdemand=np.append(Qdemand,np.count_nonzero(KitkatsTable.column('GTKitkat') <= 1.5))
# Qdemand=np.append(Qdemand,np.count_nonzero(KitkatsTable.column('GTKitkat') <= 1.25))
# Qdemand=np.append(Qdemand,np.count_nonzero(KitkatsTable.column('GTKitkat') <= 1))
# Qdemand=np.append(Qdemand,np.count_nonzero(KitkatsTable.column('GTKitkat') <= 0.75))
# Qdemand=np.append(Qdemand,np.count_nonzero(KitkatsTable.column('GTKitkat') <= 0.5))
# Qdemand=np.append(Qdemand,np.count_nonzero(KitkatsTable.column('GTKitkat') <= 0.25))
# Qdemand

In [ ]:
# Combine the two into a single table
DemandGTK=DemandGTK.with_columns('Qdemand',Qdemand)
DemandGTK

In [ ]:
DemandGTK.plot("Qdemand", "priceGTK")
plt.xlabel('Quantity')
plt.ylabel('Price')
plt.title('Demand for Green Tea Kitkats');

In [ ]:
#What are the slope and intercept of the fit line
std_units = lambda a: (a - np.mean(a)) / np.std(a)
corr = lambda x, y: np.mean(std_units(x) * std_units(y))
slope = lambda x, y: corr(x, y) * np.std(y) / np.std(x)
intercept = lambda x, y: np.mean(y) - slope(x, y) * np.mean(x)

In [ ]:
slope(DemandGTK["Qdemand"], DemandGTK["priceGTK"])

In [ ]:
intercept(DemandGTK["Qdemand"], DemandGTK["priceGTK"])

##  Let's try again for Burritos 

In [ ]:
BurritosTable=DemandTable.select('Burrito')
BurritosTable


In [ ]:
DemandBurr= Table().with_columns([
    'priceBurr',[2.5, 5, 7.5, 10.00, 12.5, 15,17.5,20],    
])
DemandBurr

In [ ]:
# Qdemand=np.count_nonzero(BurritosTable.column('Burrito') < 20)
# Qdemand=np.append(Qdemand,np.count_nonzero(BurritosTable.column('Burrito') < 17.5))
# Qdemand=np.append(Qdemand,np.count_nonzero(BurritosTable.column('Burrito') < 15))
# Qdemand=np.append(Qdemand,np.count_nonzero(BurritosTable.column('Burrito') < 12.5))
# Qdemand=np.append(Qdemand,np.count_nonzero(BurritosTable.column('Burrito') < 10))
# Qdemand=np.append(Qdemand,np.count_nonzero(BurritosTable.column('Burrito') < 7.5))
# Qdemand=np.append(Qdemand,np.count_nonzero(BurritosTable.column('Burrito') < 5))
# Qdemand=np.append(Qdemand,np.count_nonzero(BurritosTable.column('Burrito') < 2.5))
# Qdemand
Qdemand = np.flip(np.cumsum(np.flip(BurritosTable.group("Burrito").column("count"))))
if "20" not in BurritosTable.column("Burrito"):
    Qdemand = np.append(Qdemand, 0)

In [ ]:
DemandBurr=DemandBurr.with_columns('Qdemand',Qdemand)
DemandBurr

In [ ]:
DemandBurr.plot("Qdemand", "priceBurr")
plt.xlabel('Quantity')
plt.ylabel('Price')
plt.title('Demand for Burritos');